In [1]:
from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
import numpy as np

X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])

rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2, n_repeats=2, random_state=0)

for train_index, test_index in rmskf.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

TRAIN: [0 3 4 6] TEST: [1 2 5 7]
TRAIN: [1 2 5 7] TEST: [0 3 4 6]
TRAIN: [0 1 4 5] TEST: [2 3 6 7]
TRAIN: [2 3 6 7] TEST: [0 1 4 5]


In [2]:
from __future__ import print_function

import os
import pickle
import random

import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
from mordred import Calculator, descriptors
from rdkit import Chem

from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from tqdm import tqdm
from random import shuffle

import rdkit
import networkx as nx

# import deepsmiles
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils

In [3]:
random.seed(2023)

In [4]:
# sv_dir='/home/is/pei-ga/Sony/data_multi'
sv_dir = '/home/ubuntu/work/multi-modal/data_multi'

In [5]:
def smi_preprocessing(smi_sequence):
    splited_smis=[]
    length=[]
    end="/n"
    begin="&"
    element_table=["C","N","B","O","P","S","F","Cl","Br","I","(",")","=","#"]
    for i in range(len(smi_sequence)):
        smi=smi_sequence[i]
        splited_smi=[]
        j=0
        while j<len(smi):
            smi_words=[]            
            if smi[j]=="[":
                smi_words.append(smi[j])
                j=j+1
                while smi[j]!="]":
                    smi_words.append(smi[j])
                    j=j+1
                smi_words.append(smi[j])
                words = ''.join(smi_words)
                splited_smi.append(words)
                j=j+1

            else:
                smi_words.append(smi[j])

                if j+1<len(smi[j]):
                    smi_words.append(smi[j+1])
                    words = ''.join(smi_words)
                else:
                    smi_words.insert(0,smi[j-1])
                    words = ''.join(smi_words)

                if words not in element_table:
                    splited_smi.append(smi[j])
                    j=j+1
                else:
                    splited_smi.append(words)
                    j=j+2

        splited_smi.append(end)
        splited_smi.insert(0,begin)
        splited_smis.append(splited_smi)
    return splited_smis


def smi2id(smiles,vocalbulary):
    sequence_id=[]
    for i in range(len(smiles)):
        smi_id=[]
        for j in range(len(smiles[i])):
            smi_id.append(vocalbulary.index(smiles[i][j]))
        sequence_id.append(smi_id)
    return sequence_id

In [6]:
labels = pd.read_csv(f"{sv_dir}/labels.csv")
smiles = labels[['cid','smiles']]
dict_CID_smile = dict(zip(smiles['cid'],smiles['smiles']))

y = labels.iloc[:,3:].to_numpy()

y.shape

X_smiles=[]
for i in smiles['smiles']:
    #X_smiles.append(converter.encode(i))
    X_smiles.append(i)
len(X_smiles)


# maximum random
random_smiles=[]
dupli_smiles = np.array(X_smiles*100).reshape(-1,5595).T.flatten()
for i in tqdm(dupli_smiles):
    randomized_fragments = []
    for fragment in i.split('.'):
        mol = Chem.MolFromSmiles(fragment)
        randomized_fragments.append(Chem.MolToSmiles(mol, doRandom=True, canonical=False))
    shuffle(randomized_fragments)
    structure = '.'.join(randomized_fragments)
    while structure in set(random_smiles):
        for fragment in i.split('.'):
            mol = Chem.MolFromSmiles(fragment)
            randomized_fragments.append(Chem.MolToSmiles(mol, doRandom=True, canonical=False))
        shuffle(randomized_fragments)
        structure = '.'.join(randomized_fragments)
    else:
        random_smiles.append(structure)


# after_smiles=[]
# for i in random_smiles:
#     after_smiles.append(converter.encode(i))
    
after_smiles=random_smiles

smi=smi_preprocessing(after_smiles)
print(smi[0])


vocalbulary=[]
for i in smi:
    vocalbulary.extend(i)
vocalbulary=list(set(vocalbulary))

print(vocalbulary, len(vocalbulary))

docs = dict(zip(vocalbulary, range(len(vocalbulary))))
print(docs)

val_id=smi2id(smi,vocalbulary)
print(val_id[0])

tensor_X = [torch.tensor(i) for i in val_id]
tensor_X = rnn_utils.pad_sequence(tensor_X, batch_first=True)

tensor_X.shape

X = tensor_X.reshape(-1).reshape(-1,1660)
X = X.reshape(5595,10,-1)

X = np.asarray(X,dtype=object)

X_sequence = X.transpose(0,2,1)

print(X_sequence.shape,y.shape)

X_sequence

  3%|▎         | 16280/559500 [00:09<05:32, 1633.56it/s]


KeyboardInterrupt: 

In [ ]:
X_sequence[0]

array([[26, 26, 26, ..., 26, 26, 26],
       [38, 38, 38, ..., 38, 38, 38],
       [44, 38, 38, ..., 38, 44, 35],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [ ]:
#### graph ####

In [ ]:
def qm9_edges(g):
    remove_edges = []
    e={}    
    for n1, n2, d in g.edges(data=True):
        e_t = []
        # Raw distance function
        if d['b_type'] is None:
            remove_edges += [(n1, n2)]
        else:
            #e_t.append(d['distance'])
            e_t += [int(d['b_type'] == x) for x in [rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                    rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC]]
        if e_t:
            e[(n1, n2)] = e_t
    for edg in remove_edges:
        g.remove_edge(*edg)    
    
    return nx.to_numpy_matrix(g), e
    
def qm9_nodes(g, hydrogen=False):
    h = []
    for n, d in g.nodes(data=True): 
        h_t = []
        # Atom type (One-hot H, C, N, O F)
        h_t += [int(d['a_type'] == x) for x in ['H', 'C', 'N', 'O', 'F']]
        # Atomic number
        h_t.append(d['a_num'])
        # Acceptor
        h_t.append(d['acceptor'])
        # Donor
        h_t.append(d['donor'])
        # Aromatic
        h_t.append(int(d['aromatic']))
        # If number hydrogen is used as a
        if hydrogen:
            h_t.append(d['num_h'])
        h.append(h_t)
    return h

def xyz_graph_reader(CID):
    smiles = dict_CID_smile[CID]
    m = Chem.MolFromSmiles(smiles)
    m = Chem.AddHs(m)
   
    g = nx.Graph()
    # Create nodes
    for i in range(0, m.GetNumAtoms()):
        atom_i = m.GetAtomWithIdx(i)

        g.add_node(i, a_type=atom_i.GetSymbol(), a_num=atom_i.GetAtomicNum(), acceptor=0, donor=0,
                   aromatic=atom_i.GetIsAromatic(), hybridization=atom_i.GetHybridization(),
                   num_h=atom_i.GetTotalNumHs())


    # Read Edges
    for i in range(0, m.GetNumAtoms()):
        for j in range(0, m.GetNumAtoms()):
            e_ij = m.GetBondBetweenAtoms(i, j)
            if e_ij is not None:
                g.add_edge(i, j, b_type=e_ij.GetBondType())
            else:
                # Unbonded
                g.add_edge(i, j, b_type=None)
    
    l = labels[labels['cid'] ==CID]
    l = l.iloc[:,3:].values       
                
    return g , l

class Qm9():
    # Constructor
    def __init__(self, idx, vertex_transform=qm9_nodes, edge_transform=qm9_edges,
                 target_transform=None, e_representation='raw_distance'):
        self.idx = idx
        self.vertex_transform = vertex_transform
        self.edge_transform = edge_transform
        self.target_transform = target_transform
        self.e_representation = e_representation

    def __getitem__(self,index):
        
        g, target = xyz_graph_reader(self.idx[index])
        if self.vertex_transform is not None:
            h = self.vertex_transform(g)

        if self.edge_transform is not None:
            g, e = self.edge_transform(g)

#         if self.target_transform is not None:
#             target = self.target_transform(target)

        #g：adjacent matrix
        #h：node properties（list of list）
        #e：diction，key:edge，value:properties
        return (g, h, e)
        #return (g, h, e), target

    def __len__(self):
        return len(self.idx)

#     def set_target_transform(self, target_transform):
#         self.target_transform = target_transform

In [ ]:
data = Qm9(smiles['cid'].values, edge_transform=qm9_edges, e_representation="raw_distance")
feat = []
for i in range(len(data)):
    feat.append(data[i][0])
X_graph= np.asarray(data,dtype=object)

In [ ]:
#### numeric ####

In [ ]:
X_numeric = pd.read_csv(f"{sv_dir}/mordred_prune.csv")
del X_numeric["smiles"]

In [ ]:
X_numeric = X_numeric.values

In [ ]:
print(X_sequence.shape, type(X_sequence),X_graph.shape,type(X_graph),X_numeric.shape,type(X_numeric),y.shape)

(5595, 166, 10) <class 'numpy.ndarray'> (5595, 3) <class 'numpy.ndarray'> (5595, 1388) <class 'numpy.ndarray'> (5595, 91)


In [ ]:
result = []
for i in range(5595):
    #sample_numeric = list(X_numeric.values[i])
    #sample = [X_sequence[i],X_graph[i][0],X_graph[i][1],X_graph[i][2],sample_numeric]
    sample = [X_graph[i][0],X_graph[i][1],X_graph[i][2],X_numeric[i],X_sequence[i]]
    result.append(sample)

In [ ]:
X = np.asarray(result,dtype=object)

In [ ]:
rmskf = RepeatedMultilabelStratifiedKFold(n_splits=5, n_repeats=3)
for i, (train_index, test_index) in enumerate(rmskf.split(X, y)):

    sv_path = f"{sv_dir}/split_{i}.pkl"

    if os.path.exists(sv_path):
        continue

    print("TRAIN:", train_index, "TEST:", test_index)
    print("Len TRAIN:", len(train_index), "TEST:", len(test_index))

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # check label distribution
    #print(label_dist((y_train)))
    #print(label_dist((y_test)))

    data = {}
    data["X_train"] = X_train
    data["X_test"] = X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["train_index"] = train_index
    data["test_index"] = test_index

    with open(sv_path, "wb") as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

TRAIN: [   0    1    2 ... 5589 5592 5594] TEST: [   8   15   16 ... 5590 5591 5593]
Len TRAIN: 4476 TEST: 1119
TRAIN: [   1    2    3 ... 5592 5593 5594] TEST: [   0    5    9 ... 5580 5588 5589]
Len TRAIN: 4485 TEST: 1110
TRAIN: [   0    1    2 ... 5592 5593 5594] TEST: [   3   11   19 ... 5574 5577 5581]
Len TRAIN: 4478 TEST: 1117
TRAIN: [   0    3    4 ... 5590 5591 5593] TEST: [   1    2    6 ... 5579 5592 5594]
Len TRAIN: 4460 TEST: 1135
TRAIN: [   0    1    2 ... 5592 5593 5594] TEST: [   4    7   10 ... 5584 5586 5587]
Len TRAIN: 4481 TEST: 1114
TRAIN: [   0    1    3 ... 5592 5593 5594] TEST: [   2    6    8 ... 5583 5586 5589]
Len TRAIN: 4493 TEST: 1102
TRAIN: [   0    1    2 ... 5590 5591 5592] TEST: [   3   15   17 ... 5584 5593 5594]
Len TRAIN: 4478 TEST: 1117
TRAIN: [   0    1    2 ... 5592 5593 5594] TEST: [   5    7   16 ... 5585 5590 5591]
Len TRAIN: 4463 TEST: 1132
TRAIN: [   0    1    2 ... 5592 5593 5594] TEST: [  10   11   12 ... 5582 5587 5588]
Len TRAIN: 4507 TES